# 문서 군집화

In [ ]:
from konlpy.tag import Komoran
from soyclustering import SphericalKMeans
from wordcloud import WordCloud

from collections import Counter
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfTransformer
  
import matplotlib.pyplot as plt 
import nltk

## 네이버 데이터 읽기

In [ ]:
# 데이터 읽기: p77






## 형태소 분석 및 저빈도 단어 제거

In [ ]:
# Komoran()를 활용한 형태소 분석, 명사 추출 및 카운터 생성: p78






In [ ]:
# 저빈도 단어 제거






In [ ]:
noun_dict

## 커스텀 토크나이저 정의

In [ ]:
# 커스텀 토크나이저 정의
def custom_tokenizer(doc):
    return [word for word in komoran.nouns(doc) if word in noun_dict]

In [ ]:
# 커스텀 토크나이저를 이용해서 명사 시험 추출: p79




## 문서 벡터 생성

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# 벡터 생성을 위한 Counter 토크나이저 생성
vectorizer = CountVectorizer(
    tokenizer=custom_tokenizer,
    min_df=0.005,
    max_df=0.98
)

In [ ]:
# 텍스트 토크나이저 적용을 통한 벡터 생성


In [ ]:
# 벡터 생성 결과 (벡터 크기)


In [ ]:
# 벡터 생성 결과


In [ ]:
# 결과 출력 



## TF-IDF 적용

In [ ]:
# TF-IDF 적용



In [ ]:
# TF-IDF 적용 결과



## 군집화 실행

In [ ]:
# K-mean 클러스터링 적용
from sklearn.cluster import KMeans

clustersize=3   # 클러스터의 수 설정

In [ ]:
kmeans = KMeans(init='random', n_clusters=clustersize, n_init=10) # K-mean 생성

In [ ]:
# K-means 적용


In [ ]:
# 클러스터링 결과 df_texts에 추가



In [ ]:
df_texts.tail()

## 클러스터별 단어 빈도수 확인

In [ ]:
# 군집마다 가장 빈도수가 높은 단어 출력
for i in range(clustersize):
    lists = df_texts[df_texts['label']==i].iloc[:,0].to_list()

    textlist = []
    for text in lists:
        textlist.extend(custom_tokenizer(text))
        
    print ('\ncluster'+str(i)+': '+ str(Counter(textlist).most_common(20)))

## 워드 클라우드로 결과 확인

In [ ]:
# 워드 클라우드 그리는 함수 정의 (빈도 고려 하지 않음)
def drawWordcloud(texts, name='wordcloud'):
    wordcloud = WordCloud(
        font_path = 'c:/windows/fonts/gulim.ttc', # 윈도우 폰트 위치
#       font_path = '/System/Library/Fonts/Supplemental/AppleGothic.ttf',  # 맥용
        background_color='white', colormap = 'Accent_r', width = 500, height = 500)

    noun_counter = Counter(  
        [noun for text in texts for noun in custom_tokenizer(text)]
    )
    
    wordcloud_words = wordcloud.generate_from_frequencies(noun_counter)

    # wordcloud 출력
    fig = plt.figure(figsize=(10, 10))  # 그림 크기 설정
    plt.title(name)
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis('off')
    plt.show()

    # wordcloud 저장
    fig.savefig(name+'.png')
    
    return

In [ ]:
# 전체 text로 결과 출력 (빈도 고려 하지 않음)


In [ ]:
# 군집 마다 가장 빈도수가 높은 단어 출력 (빈도 고려 하지 않음)
for i in range(clustersize):
    drawWordcloud(df_texts[df_texts['label']==i].iloc[:,0].to_list(), 'cluster:'+str(i))

In [ ]:
# 워드 클라우드 그리는 함수 정의 (빈도가 높은 단어 제거)
def drawWordcloud_filter(texts, name='wordcloud'):

    wordcloud = WordCloud(
        font_path = 'c:/windows/fonts/gulim.ttc', # 윈도우 폰트 위치
#        font_path = '/System/Library/Fonts/Supplemental/AppleGothic.ttf', # 맥용
        background_color='white', colormap = 'Accent_r', width = 500, height = 500)

    # wordcloud 생성
    vectorizer = CountVectorizer(
        tokenizer=custom_tokenizer,
        min_df=0.005,
        max_df=0.95
    )
    vectorizer.fit_transform(texts)
    wordcloud_words = wordcloud.generate_from_frequencies(vectorizer.vocabulary_)

    # wordcloud 출력
    fig = plt.figure(figsize=(10, 10))  # 그림 크기 설정
    plt.title(name)
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis('off')
    plt.show()

    # wordcloud 저장
    fig.savefig(name+'.png')
    
    return

In [ ]:
# 전체 text로 결과 출력 (빈도가 높은 단어 제거)


In [ ]:
# 군집 마다 가장 빈도수가 높은 단어 출력 (빈도가 높은 단어 제거)
for i in range(clustersize):
    drawWordcloud_filter(df_texts[df_texts['label']==i].iloc[:,0].to_list(), 'cluster:'+str(i))